In [1]:
# Utilities
import pandas as pd
import numpy as np
import re
import pickle

# Machine Learning
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
def review_wordlist(review, remove_stopwords=True):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
   #print(words)
    return(words)
nltk.download('popular')
from bs4 import BeautifulSoup
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
def review_sentences(review, tokenizer, remove_stopwords=True):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    print(raw_sentences)
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    #print(sentences)
    # This returns the list of lists
    return sentences
from gensim.models import word2vec
from statistics import mode
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index_to_key)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model.wv[word])
            #featureVec = np.multiply(featureVec,model[word])
            #featureVec = statistics.median(featureVec)
            print(word)
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    print(nwords) 
    return featureVec
from statistics import mode
def featureVecMethod_code(words, model_code, num_features_code):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features_code,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model_code.wv.index_to_key)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model_code.wv[word])
            #featureVec = np.multiply(featureVec,model[word])
            #featureVec = statistics.median(featureVec)
            print(word)
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    print(nwords) 
    return featureVec
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model , num_features)
        counter = counter+1
        
    return reviewFeatureVecs
# Function for calculating the average feature vector
def getAvgFeatureVecs_code(reviews, model_code, num_features_code ):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features_code),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod_code(review, model_code ,num_features_code )
        counter = counter+1
        
    return reviewFeatureVecs

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\dassh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\dassh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\dassh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\dassh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\dassh\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [105]:
L=["A network of community computer centres, linked by wireless technology, is providing a helping hand for poor farmers in Peru. The pilot scheme in the Huaral Valley, 80 kilometres north of the capital Lima, aims to offer the 6,000-strong community up-to-date information on agricultural market prices and trends. The Agricultural Information Project for Farmers of the Chancay-Huaral Valley also provides vital links between local organisations in charge of water irrigation, enabling them to coordinate their actions. The non-government organisation, Cepes (Peruvian Centre for Social Studies) led the $200,000 project, also backed by local institutions, the Education and Agriculture ministries, and European development organisations. The Board of Irrigation Users which runs the computer centres, aims to make the network self-sustainable within three years, through the cash generated by using the telecentres as internet cafes. It is a precious element in Peru's coastal areas, because it is so scarce, and therefore it is necessary to have proper irrigation systems to make the most of it, Mr Saldarriaga told the BBC News website. "]
sentences = []
print("Parsing sentences from training set")
for review in L:
    #print (review)
    #print("\n")
    sentences += review_sentences(review, tokenizer)

L_summary=["A network of community computer centres, linked by wireless technology, is providing a helping hand for poor farmers in Peru. The pilot scheme in the Huaral Valley, 80 kilometres north of the capital Lima, aims to offer the 6,000-strong community up-to-date information on agricultural market prices and trends. The Agricultural Information Project for Farmers of the Chancay-Huaral Valley also provides vital links between local organisations in charge of water irrigation, enabling them to coordinate their actions. The non-government organisation, Cepes (Peruvian Centre for Social Studies) led the $200,000 project, also backed by local institutions, the Education and Agriculture ministries, and European development organisations. The plan includes training on computers and internet skills for both operators and users of the system, said Carlos Saldarriaga, technical coordinator at Cepes. The Board of Irrigation Users which runs the computer centres, aims to make the network self-sustainable within three years, through the cash generated by using the telecentres as internet cafes. One of the key elements of the project is the Agricultural Information System, with its flagship huaral. The system also helps the inhabitants of the Chancay-Huaral Valley to organise their vital irrigation systems. It is a precious element in Peru`s coastal areas, because it is so scarce, and therefore it is necessary to have proper irrigation systems to make the most of it, Mr Saldarriaga told the BBC News website. The information network also allows farmers to look beyond their own region, and share experiences with other colleagues from the rest of Peru and even around the world. Cepes says the involvement of the farmers has been key in the project's success. Throughout the last three years, the people have provided a vital thrust to the project; they feel it belongs to them, said Mr Saldarriaga. So far, the Huaral programme promoters say the experience has been very positive, and are already planning on spreading the model among other farmers' organisations in Peru. The Cepes researcher recalls what happened in Cuyo, a 50-family community with no electricity, during the construction of the local telecentre site. It was already dark when the technicians realised they didn't have any light bulbs to test the generator, so they turned up to the local store to buy light bulbs, recalls Carlos Saldarriaga."]
   
sentences_code=[]  
for review in L_summary:
    sentences_code += review_sentences(review, tokenizer)
    # Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Creating the model and setting values for the various parameters
num_features = 500  # Word vector dimensionality
min_word_count = 5 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10       # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

num_features_code = 500  # Word vector dimensionality
min_word_count_code = 6 # Minimum word count
num_workers_code = 4     # Number of parallel threads
context_code = 10        # Context window size
downsampling_code = 1e-3 # (0.001) Downsample setting for frequent words


Parsing sentences from training set
['A network of community computer centres, linked by wireless technology, is providing a helping hand for poor farmers in Peru.', 'The pilot scheme in the Huaral Valley, 80 kilometres north of the capital Lima, aims to offer the 6,000-strong community up-to-date information on agricultural market prices and trends.', 'The Agricultural Information Project for Farmers of the Chancay-Huaral Valley also provides vital links between local organisations in charge of water irrigation, enabling them to coordinate their actions.', 'The non-government organisation, Cepes (Peruvian Centre for Social Studies) led the $200,000 project, also backed by local institutions, the Education and Agriculture ministries, and European development organisations.', 'The Board of Irrigation Users which runs the computer centres, aims to make the network self-sustainable within three years, through the cash generated by using the telecentres as internet cafes.', "It is a precio

In [106]:
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          vector_size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling,
                          epochs=100)
model_code = word2vec.Word2Vec(sentences_code,\
                          workers=num_workers_code,\
                          vector_size=num_features_code,\
                          min_count=min_word_count_code,\
                          window=context_code,
                          sample=downsampling_code,
                          epochs=100)
# To make the model memory efficient
model.init_sims(replace=True)
model_code.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model_code_name = "300features_40minwords_10context_code"
model.save(model_name)
model_code.save(model_code_name)


2023-05-16 19:32:52,985 : INFO : collecting all words and their counts
2023-05-16 19:32:52,985 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-05-16 19:32:52,986 : INFO : collected 90 word types from a corpus of 108 raw words and 6 sentences
2023-05-16 19:32:52,986 : INFO : Creating a fresh vocabulary
2023-05-16 19:32:52,987 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 0 unique words (0.0%% of original 90, drops 90)', 'datetime': '2023-05-16T19:32:52.987475', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}
2023-05-16 19:32:52,987 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 leaves 0 word corpus (0.0%% of original 108, drops 108)', 'datetime': '2023-05-16T19:32:52.987475', 'gensim': '4.1.2', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10

Training model....


RuntimeError: you must first build vocabulary before training the model

In [107]:
clean_train_reviews = []
for review in L:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))

print(clean_train_reviews)  
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
clean_train_reviews_code = []
for review in L_summary:
    clean_train_reviews_code.append(review_wordlist(review, remove_stopwords=True))

print(clean_train_reviews_code)  
trainDataVecs_code = getAvgFeatureVecs_code(clean_train_reviews_code, model_code, num_features_code)

[['network', 'community', 'computer', 'centres', 'linked', 'wireless', 'technology', 'providing', 'helping', 'hand', 'poor', 'farmers', 'peru', 'pilot', 'scheme', 'huaral', 'valley', 'kilometres', 'north', 'capital', 'lima', 'aims', 'offer', 'strong', 'community', 'date', 'information', 'agricultural', 'market', 'prices', 'trends', 'agricultural', 'information', 'project', 'farmers', 'chancay', 'huaral', 'valley', 'also', 'provides', 'vital', 'links', 'local', 'organisations', 'charge', 'water', 'irrigation', 'enabling', 'coordinate', 'actions', 'non', 'government', 'organisation', 'cepes', 'peruvian', 'centre', 'social', 'studies', 'led', 'project', 'also', 'backed', 'local', 'institutions', 'education', 'agriculture', 'ministries', 'european', 'development', 'organisations', 'board', 'irrigation', 'users', 'runs', 'computer', 'centres', 'aims', 'make', 'network', 'self', 'sustainable', 'within', 'three', 'years', 'cash', 'generated', 'using', 'telecentres', 'internet', 'cafes', 'prec

In [108]:
trainDataVecs_array=trainDataVecs[0]
trainDataVecs_code_array=trainDataVecs_code[0]
#print(trainDataVecs_array)
#print(trainDataVecs_code_array)
unit_vector_1 = trainDataVecs_array / np.linalg.norm(trainDataVecs_array)
#print(unit_vector_1)
unit_vector_2 = trainDataVecs_code_array / np.linalg.norm(trainDataVecs_code_array)
#print(unit_vector_2)
dot_product = np.dot(unit_vector_1, unit_vector_2)
print(dot_product)
angle = np.arccos(dot_product)
print(angle)
print(np.rad2deg(angle))

0.93885875
0.35149583
20.139227
